In [1]:
%azure.connect "/subscriptions/edd613b3-8845-4654-ab8a-43c1712ec7ce/resourceGroups/AzureQuantum/providers/Microsoft.Quantum/Workspaces/iQuHack2023-workspace" location="eastus"

Authenticated using Azure.Identity.AzureCliCredential


Connected to Azure Quantum workspace iQuHack2023-workspace in location eastus.


Target ID,Current Availability,Average Queue Time (Seconds)
ionq.qpu,Available,162189
ionq.qpu.aria-1,Available,108582
ionq.simulator,Available,2
microsoft.estimator,Available,0
quantinuum.hqs-lt-s1,Available,172805
quantinuum.hqs-lt-s1-apival,Available,2
quantinuum.hqs-lt-s2,Unavailable,0
quantinuum.hqs-lt-s2-apival,Available,0
quantinuum.hqs-lt-s1-sim,Available,453
quantinuum.hqs-lt-s2-sim,Available,150


In [2]:
operation SetQubitState(desired : Result, target : Qubit) : Unit {
    if desired != M(target) {
        X(target);
    }
}

SetQubitState

In [3]:
operation TestBellState(count : Int, initial : Result) : (Int, Int, Int, Int) {
    mutable numOnesQ1 = 0;
    mutable numOnesQ2 = 0;

    // allocate the qubits
    use (q1, q2) = (Qubit(), Qubit());   
    for test in 1..count {
        SetQubitState(initial, q1);
        SetQubitState(Zero, q2);

        H(q1);                // Add the H operation after initialization and before measurement
        // CNOT(q1, q2);                   // added CNOT operation
        Controlled X([q1], q2);                   // added CONTROLLED-X operation
        
        // measure each qubit
        let resultQ1 = M(q1);            
        let resultQ2 = M(q2);           

        // Count the number of 'Ones':
        if resultQ1 == One {
            set numOnesQ1 += 1;
        }
        if resultQ2 == One {
            set numOnesQ2 += 1;
        }
    }

    // reset the qubits
    SetQubitState(Zero, q1);             
    SetQubitState(Zero, q2);
    

    // Return number of |0> states, number of |1> states
    Message("q1:Zero, One  q2:Zero, One");
    return (count - numOnesQ1, numOnesQ1, count - numOnesQ2, numOnesQ2 );

}

TestBellState

In [6]:
%simulate TestBellState count=1000 initial=1

q1:Zero, One  q2:Zero, One


(502, 498, 502, 498)

In [7]:
%simulate TestBellState count=1000 initial=0

q1:Zero, One  q2:Zero, One


(516, 484, 516, 484)